# Traditional RAG with Neo4J

In this notebook, we are using tradtional RAG strategy consisting of `Chunk` nodes arranged into linked lists.


1. Extract text from Markdown document, split text into chunks, create `Chunk` nodes
2. Enhance each `Chunk` node with a text embedding
3. Expand the `Chunk` nodes with `NEXT` relationships to form linked lists

```cypher
(:Chunk 
  chunkId: string
  text: string
  header1: string
  header2: string
  header3: string
  header4: string
  path: string
  documentUri: string
  ebmbedding: float[]
)
```

```cypher
(:Chunk)-[:NEXT]->(:Chunk)
```

## Setup

Import some python packages, set up global constants, and create a connection to the Neo4j database.

In [71]:
%run 'shared.ipynb'

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
Connecting to Neo4j at bolt://neo4j-1:7687 as neo4j
Using data from /home/jovyan/data/single
Embedding with ollama using mxbai-embed-large
Chatting with ollama using llama3


## Prepare a GraphDatabase interface

You will use the Neo4j `GraphDatabase` interface to send queries to the Neo4j database.

In [72]:
# Expect `gdb` to be defined in the shared notebook
# gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

result = gdb.execute_query("RETURN 'Hello, World!' AS message")

result.records[0].get('message')

'Hello, World!'

# Input data pre-preprocessing

The IIHF pdf document we will be working with has been preprocessed from the original source into markdown. We will use the markdown here.


# Step by step inspection of the document

### Start with the file

Get the the file name and then loading the markdown.

In [73]:
loader = DirectoryLoader('/data-transfer/iihf', glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()

print (documents[0].metadata["source"])
print (len(documents))

/data-transfer/iihf/rulebook.md
1


### Text splitter from Langchain

We can use a text splitter function from Langchain.

The `RecursiveCharacterTextSplitter` will use newlines
and then whitespace characters to break down a text until
the chunks are small enough. This strategy is generally
good at keeping paragraphs together.

Set a chunk size of 600 characters,
with 0 characters of overlap between each chunk,
using the built-in `len` function to calculate the 
text length.


In [74]:
# Splitting text into chunks using the RecursiveCharacterTextSplitter 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)

### Text splitter demonstration

You can see what the text splitter will do by splitting up
the `page_content`.

In [75]:
text_chunks = text_splitter.split_text(documents[0].page_content)
text_chunks[19]


'The Goal posts shall be of an approved design and material, extending vertically 1.22 m above the surface of the ice and set 1.83 m apart measured from the inside of the posts. A crossbar of the same material as the Goal posts shall extend from the top of one post to the top of the other. The Goal posts and crossbar shall be painted in red color and all other exterior surfaces shall be painted in white color.\n\n#### 2.2 GOAL NETS'

### Markdown Header Text Splitter combined Recursive Character Text Splitter from Langchain


#### Markdown Header Text Splitter
We first use the Markdown Header Text splitter to split on the structure of the markdown document (using Header 1 - 4).

In [77]:
print (documents[0].metadata["source"])
print (len(documents))

headers_to_split_on = [
    ("#", "header1"),
    ("##", "header2"),
    ("###", "header3"),
    ("####", "header4"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
md_header_splits = markdown_splitter.split_text(documents[0].page_content)

/data-transfer/iihf/rulebook.md
1


In [17]:
md_header_splits[0]

Document(page_content='No matter where ice hockey is played, the object of the game is the same – to put the puck into the opponent’s goal. Beyond that, ice hockey across the globe is subject to certain variations. This makes the rules of the game extremely important. These rules must be followed all times, in all countries, in all age categories, for the game to be enjoyed by everyone.  \nHockey’s speed is one of the qualities that makes it so exciting. But this skill and excitement must be balanced with fair play and respect.  \nIt is, therefore, important to make a clear separation between the purpose of all the elements of the game and to use these respectfully. These distinctions can be taught at an early age or whenever one begins to show interest in the game. And this is why hockey development begins with parents and coaches, those people most influential in guiding a person, old or young, into playing the game properly and within the rules.  \nThe IIHF Championship program enco

In [18]:
md_header_splits[11]

Document(page_content='Each Rink must have two (2) “Goal Nets”, one at either end of the Rink. The “Goal Net” is comprised of a Goal frame and netting. The open end of the goal net must face Center ice.  \nEach Goal Net must be located in the center of the Goal Line at either end and must be installed in such manner as to remain stationary during the progress of the game. The Goal posts must be kept in position by means of flexible pegs affixed in the ice or floor, but which displace the Goal Net from its moorings upon significant contact.  \nThe holes for the goal pegs must be located exactly on the Goal Line.  \nThe Goal posts shall be of an approved design and material, extending vertically 1.22 m above the surface of the ice and set 1.83 m apart measured from the inside of the posts. A crossbar of the same material as the Goal posts shall extend from the top of one post to the top of the other. The Goal posts and crossbar shall be painted in red color and all other exterior surface

#### Recursive Character Text Splitter

and now also use the Recursive Character Text Splitter to further split the text blocks.

In [78]:
# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 600
chunk_overlap = 0
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function = len,
    is_separator_regex = False,    
)

# Split
chunks = text_splitter.split_documents(md_header_splits)
chunks[0]

Document(page_content='No matter where ice hockey is played, the object of the game is the same – to put the puck into the opponent’s goal. Beyond that, ice hockey across the globe is subject to certain variations. This makes the rules of the game extremely important. These rules must be followed all times, in all countries, in all age categories, for the game to be enjoyed by everyone.  \nHockey’s speed is one of the qualities that makes it so exciting. But this skill and excitement must be balanced with fair play and respect.', metadata={'header1': 'IIHF Official Rulebook 2023/24', 'header2': 'Welcome'})

In [12]:
chunks[16]

Document(page_content='**Face-off Spot and Circle at Center Ice:**  \nA circular blue spot, 30 cm in diameter, shall be marked exactly in the center of the Rink. This spot shall be referred to as the “Center Ice Face-off Spot”. With this spot as a center, a circle of 4.50 m radius shall be marked with a blue line 5 cm wide.  \n**Face-off Spots in the Neutral Zone:**  \nTwo (2) red spots, 60 cm in diameter, shall be marked on the ice in the Neutral Zone 1.50 m from each Blue Line. These four (4) spots shall be referred to as the “Neutral-zone Face-off Spots”.', metadata={'header1': 'IIHF Official Rulebook 2023/24', 'header2': 'SECTION 01 PLAYING AREA', 'header3': 'RULE 1 RINK', 'header4': '1.9 FACE-OFF SPOTS AND CIRCLES'})

In [22]:
len(chunks)

271

## Create a graph from the chunks

You now have chunks prepared for creating a knowledge graph.

The graph will have 1 node per chunk, containing the chunk text and metadata as properties.

### Merge chunk query

You will use a Cypher query to merge the chunks into the graph.

This query accepts a query parameter called `chunkParam` which is expected
to have the data record containing the chunk and metadata.

The `MERGE` query will first match an existing node with the same `chunkId` property.

If no such node exists, it will create a new node and the `ON CREATE` clause will set the properties using values from the `chunkParam` query parameter.

In [79]:
merge_chunk_node_query = """
MERGE(c:Chunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET 
        c.source = $chunkParam.source, 
        c.chunkSeqId = $chunkParam.chunkSeqId, 
        c.path = $chunkParam.path,
        c.text = $chunkParam.text,
        c.documentUri = $chunkParam.documentUri,
        c += $chunkParam.metadata
RETURN c
"""

In [80]:
# Helper function to create nodes for all chunks.
# This will use the `merge_chunk_node_query` to create a `:Chunk` node for each chunk.
def create_chunk_id(metadata, idx) -> str:
    id = metadata["header1"]
    if 'header2' in metadata:
        id = id + '|' + metadata["header2"]
    if 'header3' in metadata:
        id = id + '|' + metadata["header3"]
    if 'header4' in metadata:
        id = id + '|' + metadata["header4"]
    id = id + '|' + str(idx)    
    return hashlib.sha1(id.encode()).hexdigest()

def create_path(metadata) -> str: 
    path = metadata["header1"]
#    if 'header2' in metadata:
#        path = path + '/' + metadata["header2"]
#    if 'header3' in metadata:
#        path = path + '/' + metadata["header3"]
#    if 'header4' in metadata:
#        path = path + '/' + metadata["header4"]
    return path.replace(' ', '_').replace('.', '_').lower()

def create_nodes_for_all_chunks(documentUri, chunks):
    node_count = 0
    for i, chunk in enumerate(chunks):
        chunk_id = create_chunk_id(chunk.metadata, i)
        path = create_path(chunk.metadata)
        gdb.execute_query(merge_chunk_node_query, 
                chunkParam = { "chunkId": chunk_id, "source":"", "chunkSeqId":i, "text": chunk.page_content, "metadata": chunk.metadata, "path": path, "documentUri": documentUri }
        )
        node_count += 1
    print(f"Created {node_count} nodes")

### Prepare unique constraint

Before calling the helper function to create a knowledge graph,
we will take one extra step to make sure we don't duplicate data.

The uniqueness constraint is also index. It's job is to ensure that
a particular property is unique for all nodes that share a common label.



In [81]:
# Create a uniqueness constraint on the chunkId property of Chunk nodes 
gdb.execute_query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS 
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
""")

created_indexes = gdb.execute_query('SHOW CONSTRAINTS').records
print(created_indexes)

[<Record id=8 name='unique_chunk' type='UNIQUENESS' entityType='NODE' labelsOrTypes=['Chunk'] properties=['chunkId'] ownedIndex='unique_chunk' propertyType=None>]


### Create index

To speed up lookup on the "path" property, we create an index


In [82]:
# Create a uniqueness constraint on the chunkId property of Chunk nodes 
gdb.execute_query("""
CREATE INDEX FOR (c:Chunk) ON (c.path)
""")

created_indexes = gdb.execute_query('SHOW INDEXES').records
print(created_indexes)

[<Record id=9 name='index_2bc8b8e7' state='POPULATING' populationPercent=66.5999984741211 type='RANGE' entityType='NODE' labelsOrTypes=['Chunk'] properties=['path'] indexProvider='range-1.0' owningConstraint=None lastRead=None readCount=None>, <Record id=7 name='unique_chunk' state='ONLINE' populationPercent=100.0 type='RANGE' entityType='NODE' labelsOrTypes=['Chunk'] properties=['chunkId'] indexProvider='range-1.0' owningConstraint='unique_chunk' lastRead=None readCount=0>]


## Load all chunks

Perform the node creation for all files in an import directory. 

In [83]:
%%time

create_nodes_for_all_chunks(documents[0].metadata["source"], chunks)

# Check the number of nodes in the graph
gdb.execute_query("MATCH (c:Chunk) RETURN count(c) as chunkCount").records[0].get('chunkCount')

Created 271 nodes
CPU times: user 136 ms, sys: 27 ms, total: 163 ms
Wall time: 1.14 s


271

In [28]:
# Check the number of unique company CUSIPs (company IDs) in the graph
# Expect this to match the `uniqueCompanyCount` from the previous cell
gdb.execute_query("MATCH (c:Chunk) RETURN count(distinct(c.header4)) as uniqueHeader4Count").records[0]

<Record uniqueHeader4Count=130>

# Add vector embeddings for the text of each chunk  

## Setup

You will use the `embeddings_api` defined in `shared.ipynb` to get the vector embeddings 
for the text of each chunk. This api will use an LLM to calculate an embedding for text.

In [23]:
# A simple example of how to use the embeddings API
text_embedding = embeddings_api.embed_query("embed this text using an LLM")

print(f"{text_embedding[0:4]} ....")

# all embeddings will have the same size, which is the dimensions of the vector
vector_dimensions = len(text_embedding) 

print(f"Text embeddings will have {vector_dimensions} dimensions")

[0.38329392671585083, -0.6893543004989624, -0.26734957098960876, 0.32013076543807983] ....
Text embeddings will have 1024 dimensions


### Prepare a vector index

Now that you have a graph populated with `Chunk` nodes, 
you can add vector embeddings.

First, prepare a vector index to store the embeddings.

The index will be called `chunks_vector` and will store
embeddings for nodes labeled as `Chunk` in a property
called `emedding`.

The embeddings index will match the dimensions of the 
embeddings returned by the `embeddings_api` and will use 
the cosine similarity function.

In [24]:
# Create a vector index called "chunks_vector" the `embedding`` property of nodes labeled `Chunk`. 
# neo4j_create_vector_index(kg, VECTOR_INDEX_NAME, 'Chunk', 'embedding')
gdb.execute_query("""
         CREATE VECTOR INDEX `chunks_vector` IF NOT EXISTS
          FOR (c:Chunk) ON (c.embedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: $vectorDimensionsParam,
            `vector.similarity_function`: 'cosine'    
         }}
""",
  vectorDimensionsParam = vector_dimensions
)

# Check the vector indexes in the graph
gdb.execute_query('SHOW VECTOR INDEXES').records

[<Record id=11 name='chunks_vector' state='ONLINE' populationPercent=100.0 type='VECTOR' entityType='NODE' labelsOrTypes=['Chunk'] properties=['embedding'] indexProvider='vector-2.0' owningConstraint=None lastRead=None readCount=None>]

### Create text embeddings

Creating the text embeddings will be a two step process. 

First, collect all chunk text and chunk ids from the graph.
Yes these are the same chunk ids that were used to create the graph
and you could save time by doing this all at once. We're doing
this incrementally to show the process, not optimized for speed.

Next, use the `embeddings_api` to get the embeddings for the text
and write those values back into the graph. 

This will take some time to run as we're doing it one chunk at a time,
calling out to the `embeddings_api` for each then writing all those
results back into the graph.

In [26]:
%%time

def text_for_embedding(text, header2, header3, header4) -> str:
    text_for_embed = text;
    if header4 is not None:
        text_for_embed = header4 + '>>' + text_for_embed
    return text_for_embed;

# Create vector embeddings for all the Chunk text, in batches.
# Use this for larger number of chunks so that the query
# can be re-run without losing all progress
print("Finding all chunks that need embedding...")
all_chunks_for_embed = gdb.execute_query("""
  MATCH (chunk:Chunk) WHERE chunk.embedding IS NULL
  RETURN chunk.text AS text, chunk.header2 as header2, chunk.header3 as header3, chunk.header4 as header4, chunk.chunkId AS chunkId
  """).records

print("Generating vector embeddings, then writing into each chunk...")
for chunk in all_chunks_for_embed:
  text = text_for_embedding(chunk['text'], chunk['header2'], chunk['header3'], chunk['header4'])
  #print (text)
  embedding = embeddings_api.embed_query(text)
  gdb.execute_query("""
    MATCH (chunk:Chunk {chunkId: $chunkIdParam})
    CALL db.create.setNodeVectorProperty(chunk, "embedding", $embeddingParam)    
    """, 
    chunkIdParam=chunk['chunkId'], embeddingParam=embedding
  )

Finding all chunks that need embedding...
Generating vector embeddings, then writing into each chunk...
CPU times: user 1.63 s, sys: 165 ms, total: 1.8 s
Wall time: 20.5 s


**This next cell is an alternative to create embeddings and the vector index in one by using Langchain**

# Example questions - vector similarity search with Neo4j

### Try Neo4j vector search helper

The `shared.ipynb` notebook has a helper function to perform a vector similarity search
using the Neo4j Knowledge Graph.

It will perform vector similarity search using the `chunks_vector` vector index.

In [ ]:
search_results = neo4j_vector_search(
#    'what is the size of the rink', VECTOR_INDEX_NAME
    'what happens if player is injured', VECTOR_INDEX_NAME
)
search_results[0]


### Question Answering chat with Langchain 

Notice that we only performed vector search. So what we're getting
back is the raw chunk text.

If we want to create a chatbot that provides actual answers to
a question, we can build a RAG system using Langchain.

In [69]:
# try the chat api directly
result = chat_api.invoke("what happens if player is injured")
#result = chat_api.invoke("what is the size of the rink")
result.content

'In the context of a fantasy sports league, if a player is injured, it can have various effects on their performance and your team\'s overall success. Here are some possible scenarios:\n\n1. **Injury Report**: The fantasy platform or league will typically provide an injury report, which indicates the severity of the injury and the expected recovery time.\n2. **Out for the Season**: If a player is severely injured and out for the season, they may be placed on the "injured reserve" (IR) list. This means they won\'t contribute to your team\'s scoring for the remainder of the season.\n3. **Short-Term Absence**: For less severe injuries, a player might miss only a few games or weeks before returning to action. In this case, you can continue to start them in your lineup once they\'re healthy enough to play.\n4. **Questionable Status**: Some players may be listed as "questionable" for a game, indicating that their status is uncertain due to the injury. You might need to make an informed decis

### Neo4j Vector Store

The easiest way to start using Neo4j with Langchain is with the `Neo4jVector` interface. This makes Neo4j look like a vector store using
the vector index you created earlier.

Under the hood, it will use the Cypher language for performing vector similarity searches.

The configuration specifies a few important things:
- use the defined `embeddings_api` for embeddings
- how to connect to the Neo4j database
- the name of the vector index to use
- the label of the nodes to search
- the property name of the text on those nodes
- and, the property name of the embeddings on those nodes

That vector store then gets converted into a retriever
and finally added to a Question Answering chain.

In [53]:
# Create a langchain vector store from the existing Neo4j knowledge graph.
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=embeddings_api,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

# RAG prompt
prompt = hub.pull("rlm/rag-prompt-llama")

# Create a retriever from the vector store
retriever = neo4j_vector_store.as_retriever(search_kwargs={'k': 1})

# Create a chatbot Question & Answer chain from the retriever
#chain = RetrievalQAWithSourcesChain.from_chain_type(
#    chat_api, chain_type="stuff", retriever=retriever
#)

chain = RetrievalQA.from_chain_type(
    chat_api, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True, 
    chain_type_kwargs={"prompt": prompt, "verbose": True}
)

chain_traditional = prettifyChain(chain)

In [54]:
docs = retriever.invoke("what happens if player is injured")
docs

[Document(page_content='\ntext: When a Player is injured so that they cannot continue play or go to their Players’ Bench, the play shall not be stopped until the injured Player’s Team has secured control of the puck. If the Player’s Team is in “control of the puck” at the time of injury, play shall be stopped immediately unless their Team is in a scoring position.', metadata={'source': '', 'path': 'iihf_official_rulebook_2023/24', 'chunkId': '9c8d74e33176f1e321d886cb3f7121a0da0e0307', 'documentUri': '/data-transfer/iihf/rulebook.md', 'chunkSeqId': 57, 'header4': '8.1. INJURED PLAYER', 'header3': 'RULE 8 INJURED PLAYERS', 'header2': 'SECTION 02 TEAMS', 'header1': 'IIHF Official Rulebook 2023/24'})]

### Ask some questions

Finally, you can use the Langchain chain, which combines the retriever
and the vector store into a nice question and answer interface.

You can see both the answer and the source that the answer came from.

In [70]:
#prettyVectorSearch("what is the size of the rink?")
chain_traditional("what happens if player is injured")



> Entering new RetrievalQA chain...


ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='192.168.1.102', port=11434): Max retries exceeded with url: /api/embeddings (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0xffff32c58310>, 'Connection to 192.168.1.102 timed out. (connect timeout=None)'))

# Expand - connect the chunks into linked lists

You can now create relationships between all nodes in that list of chunks, effectively creating a linked list from the
first chunk to the last.



In [27]:
%%time

# Collect all the form IDs and form 10k item names
distinct_path_result = gdb.execute_query("""
MATCH (c:Chunk) RETURN DISTINCT c.path as path
""").records

distinct_path_list = list(map(lambda x: x['path'], distinct_path_result))

# Connect *all* section chunks into a linked list..
cypher = """
  MATCH (from_same_path:Chunk) // match all chunks
  WHERE from_same_path.path = $path // where the chunks are from the same path
  WITH from_same_path // with those collections of chunks
    ORDER BY from_same_path.chunkSeqId ASC // order the chunks by their sequence ID
  WITH collect(from_same_path) as same_path_chunk_list // collect the chunks into a list
    CALL apoc.nodes.link(same_path_chunk_list, "NEXT", {avoidDuplicates: true}) // then create a linked list in the graph
  RETURN size(same_path_chunk_list)
"""

for path in distinct_path_list:
    gdb.execute_query(cypher, 
             path=path
    )


CPU times: user 4.9 ms, sys: 7.83 ms, total: 12.7 ms
Wall time: 99.9 ms


### Search with Query Window

You can now create a question answering chain.

The default Neo4jVector uses a basic cypher query to peform vector similarity search.

That query can be extended to do whatever you want in a Cypher.

In [56]:
# This Cypher query extension will receive two variables: node and score and it should should return three fields: text, score, and metadata.
retrieval_query_window = """
 OPTIONAL MATCH window=
    (:Chunk)-[:NEXT*0..1]->(node)-[:NEXT*0..1]->(:Chunk)
WITH node, score, window as longestWindow 
  ORDER BY node,  length(window) DESC
WITH nodes(longestWindow) as chunkList, node, score
  UNWIND chunkList as chunkRows
WITH collect(chunkRows.text) as textList, node, score
WITH apoc.text.join(textList, " \n ") as text,
    score,
    node {.source} AS metadata 
RETURN text, score, metadata  ORDER BY score DESC LIMIT 1 
"""

vector_store_window = Neo4jVector.from_existing_index(
    embedding=embeddings_api,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=retrieval_query_window
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever(search_kwargs={'k': 1})

# Create a chatbot Question & Answer chain from the retriever
chain_window = prettifyChain(RetrievalQA.from_chain_type(
    chat_api, 
    chain_type="stuff", 
    retriever=retriever_window,
    chain_type_kwargs={"verbose": True}
))

In [57]:
docs = retriever_window.invoke("what happens if player is injured")
docs

[Document(page_content='The injured Player must wait until their substitute has been released from the Penalty Box before they are eligible to play. If, however, there is a stoppage of play prior to the expiration of their penalty, they must then replace their Teammate in the Penalty Box and is then eligible to return once their penalty has expired. \n When a Player is injured so that they cannot continue play or go to their Players’ Bench, the play shall not be stopped until the injured Player’s Team has secured control of the puck. If the Player’s Team is in “control of the puck” at the time of injury, play shall be stopped immediately unless their Team is in a scoring position. \n In the case where it is obvious that a Player has sustained a serious injury, the Referee and/or Linesperson may stop the play immediately. Where an injury has occurred to a Player and there is a stoppage of play, a Team Doctor (or other Medical Personnel) may go onto the ice to attend to the injured Playe

In [58]:
chain_window("what happens if player is injured")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
The injured Player must wait until their substitute has been released from the Penalty Box before they are eligible to play. If, however, there is a stoppage of play prior to the expiration of their penalty, they must then replace their Teammate in the Penalty Box and is then eligible to return once their penalty has expired. 
 When a Player is injured so that they cannot continue play or go to their Players’ Bench, the play shall not be stopped until the injured Player’s Team has secured control of the puck. If the Player’s Team is in “control of the puck” at the time of injury, play shall be stopped immediately unless their Team is in a scoring position. 
 In the case where it is obvious that a Pla

### Debugging

In [85]:
retrieval_query_window = """
 OPTIONAL MATCH window=
    (:Chunk)-[:NEXT*0..1]->(node)-[:NEXT*0..1]->(:Chunk)
WITH node, score, window as longestWindow 
  ORDER BY node,  length(window) DESC
WITH nodes(longestWindow) as chunkList, node, score
  UNWIND chunkList as chunkRows
WITH collect(chunkRows.text) as textList, node, score
WITH apoc.text.join(textList, " \n ") as text,
    score,
    node {.source} AS metadata 
RETURN text, score, metadata  ORDER BY score DESC LIMIT 1 
"""

def neo4j_vector_search_2(question, retrieval_query):
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    CALL db.index.vector.queryNodes($index_name, $top_k, $question_embedding) 
        YIELD node, score
  """ + retrieval_query
  similar = []

  print ("Using vector index: " + str(VECTOR_INDEX_NAME))
    
  question_embedding = embeddings_api.embed_query(question)
  return gdb.execute_query(vector_search_query,
                      question=question, 
                      question_embedding=question_embedding, 
                      index_name=VECTOR_INDEX_NAME, 
                      top_k=5
                    ).records

search_results = neo4j_vector_search_2(
    'what happens if player is injured', retrieval_query_window
)
search_results[0]
    

Using vector index: chunks_vector


<Record text='The injured Player must wait until their substitute has been released from the Penalty Box before they are eligible to play. If, however, there is a stoppage of play prior to the expiration of their penalty, they must then replace their Teammate in the Penalty Box and is then eligible to return once their penalty has expired. \n When a Player is injured so that they cannot continue play or go to their Players’ Bench, the play shall not be stopped until the injured Player’s Team has secured control of the puck. If the Player’s Team is in “control of the puck” at the time of injury, play shall be stopped immediately unless their Team is in a scoring position. \n In the case where it is obvious that a Player has sustained a serious injury, the Referee and/or Linesperson may stop the play imme- diately. Where an injury has occurred to a Player and there is a stoppage of play, a Team Doctor (or other Medical Personnel) may go onto the ice to attend to the injured Player withou

In [59]:
question_embedding = embeddings_api.embed_query('what happens if player is injured')

In [ ]:
print(question_embedding)